# Télécharger le modèle préentraîné

Tomas Chambon a eu la gentillesse de préentraîner ULMfit avec Wikipedia en Français.
- https://github.com/tchambon/deepfrench


Les poids sont disponnibles ici:
- https://drive.google.com/open?id=1_0D3zv5H7iMW1qk7wHNN1yQrHi6arv8g

In [1]:
import pathlib

p = pathlib.Path('.')

ls = lambda x : list(x.iterdir())

In [3]:
ls(p/'large_files')

[PosixPath('large_files/itosref30k.pkl'), PosixPath('large_files/wref30k.pth')]

In [4]:
deep_french_weights = p/'large_files'/'wref30k.pth'
deep_french_itos    = p/'large_files'/'itosref30k.pkl'

# Importer Fastai & Fastai.Text

In [5]:
from fastai import *
from fastai.text import *

In [7]:
Tokenizer??

Doc pour le tokenizer : https://docs.fast.ai/text.transform.html#Tokenization

Bon il semble qu'il me faille un dataset

# Trouver un dataset

_Defi Fouille de Textes_ ou *DEFT* pour les intimes https://deft.limsi.fr/


*"2007 : corpus de débats parlementaires, environ 28 000 interventions anonymisées (Hommes et partis politiques) sur des projets de lois relatifs à l'énergie. La valeur de vote (0 = contre, 1 = pour) est indiquée avec chaque document, [apprentissage](https://deft.limsi.fr/corpus/corpus_debats_deft07_apprentissage.tar.gz) [7,4 Mo], [test](https://deft.limsi.fr/corpus/corpus_debats_deft07_test.tar.gz) [4,6 Mo], [reference](https://deft.limsi.fr/corpus/corpus_debats_test_classes.xml) [1,3 Mo]"*

Plutôt intriguant, peut se porter à une tache de classification.

On va entraîner le LM (*language model*) sur train & test comme le conseil _Jeremy Howard_ puis on va séparer le *train set* en 2 80/20 entraîner un modèle de classification et regarder le résultat sur le *test set*.



In [9]:
train_xml = p/'large_files'/'corpus_debats_train.xml'
test_xml  = p/'large_files'/'corpusDebats8-test.xml'

Bon super les données sont dans du XML...

## Explorer le XML

In [16]:
lines = [line.rstrip() for line in open(train_xml, encoding='iso-8859-1')]

print(lines[:10])

['<?xml version="1.0" encoding="iso-8859-1"?>', '<!DOCTYPE CORPUS SYSTEM "deft07.dtd">', '<CORPUS>', '<DESCRIPTION>', "<TITRE>Corpus des débats parlementaires à l'Assemblée Nationale</TITRE>", '<DIFFUSION>', "<CONTACT>Comité d'organisation de DEFT'07</CONTACT>", '<CADRE>Campagne DEFT 2007 - http://deft07.limsi.fr/</CADRE>', '<DISPONIBILITE region="monde" statuts="libre">Utilisable librement à des fins non commerciales</DISPONIBILITE>', '</DIFFUSION>']


In [17]:
print(len(lines))

173030


In [18]:
print(lines[5000:5010])

['</TEXTE>', '</DOCUMENT>', '<DOCUMENT id="4:499">', '<EVALUATION nombre="1">', '<NOTE valeur="0" confiance="1.00" />', '</EVALUATION>', '<TEXTE>', '<![CDATA[', "Avant de décider du devenir de Gaz de France, ne croyez-vous pas qu'il serait sage de savoir comment la Commission européenne propose d'écrire la fin de l'histoire ? Si, au bout du bout, en décembre, les conditions posées par les actionnaires de Suez en préalable à la fusion sont inacceptables - ou inenvisageables à quelques semaines d'une élection présidentielle - et si le Gouvernement les refuse, on aura ouvert le capital de Gaz de France pour rien...", ']]>']


In [44]:
print(lines[-500:])

['<EVALUATION nombre="1">', '<NOTE valeur="0" confiance="1.00" />', '</EVALUATION>', '<TEXTE>', '<![CDATA[', "Nous en avons terminé avec l'examen des articles. Je rappelle que la Conférence des Présidents a décidé que les explications de vote et le vote par scrutin public sur l'ensemble du projet de loi de financement de la sécurité sociale pour 2005 auront lieu le mardi 2 novembre, après les questions au Gouvernement.", ']]>', '</TEXTE>', '</DOCUMENT>', '<DOCUMENT id="4:17251">', '<EVALUATION nombre="1">', '<NOTE valeur="1" confiance="1.00" />', '</EVALUATION>', '<TEXTE>', '<![CDATA[', "Les débats sur ce texte ont montré combien la santé reste une problématique chère aux Français. Si une bonne santé ne s'achète pas, l'accès à des soins de qualité y contribue à l'évidence et le Gouvernement a, cette année encore, bien compris la nécessité de mobiliser des moyens considérables pour les mettre au service de cette priorité. Pour l'essentiel, les critiques de l'opposition ont porté sur la 

# Exploration 

Essayons avec http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/


(Echec, utilisons simplement xml.etree.Elementree et faisons les choses nous même)

In [61]:
import xml.etree.ElementTree as ET
import pandas as pd

import warnings; warnings.simplefilter('ignore')

In [36]:
xml = ET.XML(' '.join(lines))

In [62]:
xml.getchildren()

[<Element 'DESCRIPTION' at 0x7f1837b355e8>,
 <Element 'DOCUMENT' at 0x7f1837b35b88>,
 <Element 'DOCUMENT' at 0x7f1837b35cc8>,
 <Element 'DOCUMENT' at 0x7f1837b35e08>,
 <Element 'DOCUMENT' at 0x7f1837b35f48>,
 <Element 'DOCUMENT' at 0x7f183c2040e8>,
 <Element 'DOCUMENT' at 0x7f183c204228>,
 <Element 'DOCUMENT' at 0x7f183c204368>,
 <Element 'DOCUMENT' at 0x7f183c2044a8>,
 <Element 'DOCUMENT' at 0x7f183c2045e8>,
 <Element 'DOCUMENT' at 0x7f183c204728>,
 <Element 'DOCUMENT' at 0x7f183c204868>,
 <Element 'DOCUMENT' at 0x7f183c2049a8>,
 <Element 'DOCUMENT' at 0x7f183c204ae8>,
 <Element 'DOCUMENT' at 0x7f183c204c28>,
 <Element 'DOCUMENT' at 0x7f183c204d68>,
 <Element 'DOCUMENT' at 0x7f183c204ea8>,
 <Element 'DOCUMENT' at 0x7f183c206048>,
 <Element 'DOCUMENT' at 0x7f183c206188>,
 <Element 'DOCUMENT' at 0x7f183c2062c8>,
 <Element 'DOCUMENT' at 0x7f183c206408>,
 <Element 'DOCUMENT' at 0x7f183c206548>,
 <Element 'DOCUMENT' at 0x7f183c206688>,
 <Element 'DOCUMENT' at 0x7f183c2067c8>,
 <Element 'DO

In [63]:
document1 = xml.getchildren()[1];

document1.getchildren();

In [104]:
for each in document1.getchildren():
    print(each)
    print("Items =", each.items())
    print("Text  =", each.text)
    print('vvv')
    for each2 in each.getchildren():
        print(each2)
        print(each2.items())
        print('-'*20)
    print('^^^')
    print('-'*20)

<Element 'EVALUATION' at 0x7f1837b35bd8>
Items = [('nombre', '1')]
Text  =  
vvv
<Element 'NOTE' at 0x7f1837b35c28>
[('valeur', '1'), ('confiance', '1.00')]
--------------------
^^^
--------------------
<Element 'TEXTE' at 0x7f1837b35c78>
Items = []
Text  =   Merci, Monsieur le Premier ministre, pour votre détermination. Vous avez écouté les Français, et vous leur avez ouvert de nouveaux horizons. Vous savez combien les parlementaires peuvent être des traits d'union entre les Français et leur gouvernement, et nous ferons tout pour répondre à l'attente vigilante de nos compatriotes. Les défis sont sérieux. Nous avons mesuré l'ampleur des crises qui secouent notre pays : crise de la démocratie, exprimée à travers un vote extrémiste, et traduisant la peur de ne pas être écouté, d'être tenu à l'écart du débat démocratique ; crise de la politique : l'impuissance d'un Etat qui n'a pas été modernisé a jeté le doute sur l'efficacité des responsables politiques. Crise enfin du sens collectif : 

In [65]:
# get all documents block, get rid of the description block
all_docs = xml.getchildren()[1:]

print(all_docs[:5])
print(all_docs[-5:])
len(all_docs)

[<Element 'DOCUMENT' at 0x7f1837b35b88>, <Element 'DOCUMENT' at 0x7f1837b35cc8>, <Element 'DOCUMENT' at 0x7f1837b35e08>, <Element 'DOCUMENT' at 0x7f1837b35f48>, <Element 'DOCUMENT' at 0x7f183c2040e8>]
[<Element 'DOCUMENT' at 0x7f18344bd1d8>, <Element 'DOCUMENT' at 0x7f18344bd318>, <Element 'DOCUMENT' at 0x7f18344bd458>, <Element 'DOCUMENT' at 0x7f18344bd598>, <Element 'DOCUMENT' at 0x7f18344bd6d8>]


17299

# Extraire les informations du XML



In [81]:
for doc in all_docs[:5]:
    note = doc.find('EVALUATION').find('NOTE')
    print(note.items())
    text = doc.find('TEXTE')
    print(text.text[:100])

[('valeur', '1'), ('confiance', '1.00')]
  Merci, Monsieur le Premier ministre, pour votre détermination. Vous avez écouté les Français, et v
[('valeur', '1'), ('confiance', '1.00')]
  Il sera plus judicieux de laisser les Français choisir le rythme et la durée de leur vie professio
[('valeur', '0'), ('confiance', '1.00')]
  Au terme d'un processus électoral de trois mois, les Français ont fait leur choix ; il s'impose à 
[('valeur', '0'), ('confiance', '1.00')]
  Vous avez renvoyé leur éventuelle révision à la négociation collective : nous serons attentifs à c
[('valeur', '0'), ('confiance', '1.00')]
  Nous marquerons, disais-je, notre attachement au service public, à la protection sociale, à la ret


In [95]:
parse_note = lambda n : (n.items()[0][1],n.items()[1][1])

def parse_docs(docs):
    data = []
    for doc in docs:
        note = doc.find('EVALUATION').find('NOTE')
        sent, conf = parse_note(note)
        text = doc.find('TEXTE').text
        data.append([text, sent, conf])
    return data
    
train_data = parse_docs(all_docs)
train_df = pd.DataFrame(train_data, columns=["Text", "Sent", "Conf"])

In [96]:
train_df.head()

,Text,Sent,Conf
0,"Merci, Monsieur le Premier ministre, pour vo...",1,1.00
1,Il sera plus judicieux de laisser les França...,1,1.00
2,Au terme d'un processus électoral de trois m...,0,1.00
3,Vous avez renvoyé leur éventuelle révision à...,0,1.00
4,"Nous marquerons, disais-je, notre attachemen...",0,1.00


In [112]:
lines = [line.rstrip() for line in open(test_xml, encoding='iso-8859-1')]
xml = ET.XML(" ".join(lines))
all_docs = xml.getchildren()[1:]

def parse_docs(docs):
    data = []
    for doc in docs:
        #note = doc.find('EVALUATION').find('NOTE')
        #sent, conf = parse_note(note)
        text = doc.find('TEXTE').text
        data.append([text]) #, sent, conf])
    return data

test_data = parse_docs(all_docs)
test_df = pd.DataFrame(test_data, columns=["Text"])

In [113]:
test_df.head()

,Text
0,La commission a rejeté cet amendement. Trois...
1,Je comprends l'esprit de cet amendement. L'<...
2,Je regrette que vous qualifiiez ainsi mon in...
3,"Le CIE est soit un CDI, soit un CDD au sens ..."
4,L'article 28 abroge l'article L. 322-4-1 du ...


# Fine-tuner le *Language Model*